In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.metrics import confusion_matrix
#plt.style.use('notebook.mplstyle')
# Speed up scikit-learn
#from sklearnex import patch_sklearn
#patch_sklearn()

In [ ]:
# Load data from https://www.openml.org/d/554
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [ ]:
# Create a figure window
fig = plt.figure(figsize=[17, 6])

# Loop over all numbers and plot an example image of each one
for i in range(10):
    # Create a subplot
    ax = fig.add_subplot(2, 5, i+1)
    # Find an image of the right number
    idx_tmp = np.where(y==str(i))[0][0]
    # Plot the number as an image
    ax.imshow(X[idx_tmp, :].reshape(28, 28), cmap=cm.Greys_r)
    ax.set(xticks=[], yticks=[])

In [ ]:
# Normalize so that all values are between 0 and 1
X = X / X[:].max()

# We use a very large test set size to speed up computations in Colab
# you can use 0.5 instead if running on your own computer
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=1)

# Scale down the test set to speed up predictions in Colab
X_test = X_test[:10000, :]
y_test = y_test[:10000]

# Cross-validation
n_folds = 2;
kf = KFold(n_splits=n_folds, shuffle=True, random_state=1)
c_values = np.logspace(-1, 2, 4)

# Loop over all C values
accuracy_per_c = []
for i in range(c_values.size):
    
    print('Trying C={:1.1e}'.format(c_values[i]))
    svc = SVC(C=c_values[i], max_iter=5000)
    
    accuracy_sum = 0
    # Loop over all division into training and validation sets
    for train_index, val_index in kf.split(X_train):
        
        # SPlit into training and validation sets
        X_cv_train, X_cv_val = X_train[train_index, :], X_train[val_index, :]
        y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]
        
        # Fit a model and compute the MSE for validation set predictions
        svc.fit(X_cv_train, y_cv_train)
        y_hat_cv_val = svc.predict(X_cv_val)
        acc_tmp = np.mean(y_cv_val == y_hat_cv_val)
        accuracy_sum += acc_tmp
        print('Validation fold accuracy: {:1.3f}'.format(acc_tmp))
                           
    # Store away the final accuracy value for each C value
    accuracy_per_c.append(accuracy_sum / n_folds)

In [ ]:
# Find the best C value 
best_idx = np.argmax(accuracy_per_c)
best_c_value = c_values[best_idx]

# Visualize the cofusion matrix
fig, ax = plt.subplots(1, 1)
ax.plot(c_values, accuracy_per_c, 'ko-')
ax.plot(c_values[best_idx], accuracy_per_c[best_idx], 'ko', ms=20, mfc='none')
ax.set(xscale='log', xlabel='C', ylabel='Accuracy', title='Best C value: {:1.1e}'.format(best_c_value));

In [ ]:
# Fit a model using the best C value
svc = SVC(C=best_c_value, verbose=1, max_iter=5000)
svc.fit(X_train, y_train)

In [ ]:
# Check the accuracy of our model
y_hat_train = svc.predict(X_train)
y_hat_test = svc.predict(X_test)
print('Train set accuracy: {:2.1f} %'.format(100*np.mean(y_hat_train == y_train)))
print('Test set accuracy: {:2.1f} %'.format(100*np.mean(y_hat_test == y_test)))

In [ ]:
# Compute the confusion matrix
conf_mat = confusion_matrix(y_test, y_hat_test)
print(conf_mat)
max_nondiag_val = np.flip(np.sort(conf_mat.flatten()))[10]

# Visualize the cofusion matrix
fig, ax = plt.subplots(1, 1, figsize=[8, 6])
ih = ax.imshow(conf_mat, vmin=0, vmax=2*max_nondiag_val, cmap=cm.Greys_r)
ax.set(xticks=range(10), yticks=range(10), xlabel='Pred. label', ylabel='True label', title='Confusion matrix')
fig.colorbar(ih, label='number of samples');

In [ ]:
# Find incorrectly predicted images
incorrect_idx = np.where(y_test != y_hat_test)[0]

# Create a figure window
fig = plt.figure(figsize=[17, 14])

# Loop over incorrectly classified images and plot examples
for i in range(25):
    # Create a subplot
    ax = fig.add_subplot(5, 5, i+1)
    # Select an incorreclty classified image
    idx_tmp = incorrect_idx[i]
    # Plot the image
    ax.imshow(X_test[idx_tmp, :].reshape(28, 28), cmap=cm.Greys_r)
    ax.set(xticks=[], yticks=[], title='True: {:}; Pred. {:}'.format(y_test[idx_tmp], y_hat_test[idx_tmp]))